In [ ]:
import os
import shutil
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import obspy
from IPython.display import Image, display

from utils.misc import print_verification
from utils.plot_all_seismograms import plot_all_seismograms
from utils.plot_ascii_trace import read_ascii_trace

In [ ]:
# Define root_dir only if it is not already defined. This way root_dir stays the same, even if you change the work directory.
try:
    root_dir
except NameError:
    root_dir = Path(os.curdir).absolute()

# SPECFEM bin directory
SPECFEM2D_BIN = root_dir / "specfem2d" / "bin"
# Simulation directory
run_folder = root_dir / "05_ASCII_model"
# Go into run directory
os.chdir(run_folder)
# More readable paths
run_folder = run_folder.relative_to(Path(".").absolute())
output_folder = run_folder / "OUTPUT_FILES"

In [ ]:
def make_output_dir():
    if output_folder.exists():
        shutil.rmtree(output_folder)
    output_folder.mkdir()

# Saving the model as ASCII file

In [ ]:
! grep SAVE_MODEL DATA/Par_file

In [ ]:
! ../utils/parfile -s NSTEP 2

In [ ]:
make_output_dir()
! $SPECFEM2D_BIN/xmeshfem2D > $output_folder/output_mesher.txt
! $SPECFEM2D_BIN/xspecfem2D > $output_folder/output_solver.txt

Outputs the `DATA/proc000000_rho_vp_vs.dat` file. Format is `X Z RHO VP VS`

In [ ]:
! head DATA/proc000000_rho_vp_vs.dat

### Modifying the Model

In [ ]:
# Puts a low velocity zone in 1000-2000 in X and Z directions.
model = np.loadtxt("./DATA/proc000000_rho_vp_vs.dat")
rows, columns = model.shape

XMIN, XMAX = 1000, 2000
ZMIN, ZMAX = 1000, 2000

for i in range(rows):
    # X Z rho vp vs
    x, z, rho, vp, vs = model[i, :]
    if XMIN <= x <= XMAX and ZMIN <= z <= ZMAX:
        # half Vp and Vs
        model[i, 3] = vp / 2.0
        model[i, 4] = vp / 2.0

np.savetxt("./DATA/proc000000_rho_vp_vs.dat", model)

### Using modified model

In [ ]:
! parfile -s MODEL ascii
! parfile -s SAVE_MODEL default
! parfile -s NSTEP 1600

## Running modified model

In [ ]:
make_output_dir()
! $SPECFEM2D_BIN/xmeshfem2D > $output_folder/output_mesher.txt
! $SPECFEM2D_BIN/xspecfem2D > $output_folder/output_solver.txt

### Wavefield Plot

In [ ]:
Image(output_folder / "forward_image000000500.jpg")

### Seismograms

In [ ]:
fig, axes = plot_all_seismograms(output_folder)
fig.tight_layout()